# Deep Learning for Automatic Labeling of CT Images
## By: Ian Pan, MD.ai modified by Anouk Stein, MD.ai and Ross Filice MD, MedStar Georgetown University Hospital to predict chest, abdomen, or pelvic slices. Note lower chest/upper abdomen may have labels for both chest and abdomen.

In [ ]:
!git clone https://github.com/rwfilice/bodypart.git

In [ ]:
!pip install pydicom

In [ ]:
from scipy.ndimage.interpolation import zoom

import matplotlib.pyplot as plt
import pydicom
import pandas as pd 
import numpy as np 
import glob
import os 
import re 
import json
from pathlib import Path

from keras.applications.imagenet_utils import preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import Model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.models import model_from_json

import tensorflow as tf 

# Set seed for reproducibility
tf.random.set_seed(88) ; np.random.seed(88) 

# For data augmentation
from albumentations import (
    Compose, OneOf, HorizontalFlip, Blur, RandomGamma, RandomContrast, RandomBrightness
)

In [ ]:
testPath = Path('bodypart/testnpy')
testList = list(sorted(testPath.glob('**/*.npy'), key=lambda fn: int(re.search('-([0-9]*)', str(fn)).group(1))))

In [ ]:
testList

In [ ]:
def get_dicom_and_uid(path_to_npy):
    '''
    Given a filepath, return the npy file and corresponding SOPInstanceUID. 
    '''
    path_to_npy = str(path_to_npy)
    dicom_file = np.load(path_to_npy)
    uid = path_to_npy.split('/')[-1].replace('.npy', '')
    return dicom_file, uid

In [ ]:
def convert_dicom_to_8bit(npy_file, width, level, imsize=(224.,224.), clip=True): 
    '''
    Given a DICOM file, window specifications, and image size, 
    return the image as a Numpy array scaled to [0,255] of the specified size. 
    '''
    array = npy_file.copy() 
    #array = array + int(dicom_file.RescaleIntercept) #we did this on preprocess
    #array = array * int(dicom_file.RescaleSlope) #we did this on preprocess
    array = np.clip(array, level - width / 2, level + width / 2)
    # Rescale to [0, 255]
    array -= np.min(array) 
    array /= np.max(array) 
    array *= 255.
    array = array.astype('uint8')
    
    if clip:
    # Sometimes there is dead space around the images -- let's get rid of that
        nonzeros = np.nonzero(array) 
        x1 = np.min(nonzeros[0]) ; x2 = np.max(nonzeros[0])
        y1 = np.min(nonzeros[1]) ; y2 = np.max(nonzeros[1])
        array = array[x1:x2,y1:y2]

    # Resize image if necessary
    resize_x = float(imsize[0]) / array.shape[0] 
    resize_y = float(imsize[1]) / array.shape[1] 
    if resize_x != 1. or resize_y != 1.:
        array = zoom(array, [resize_x, resize_y], order=1, prefilter=False)
    return np.expand_dims(array, axis=-1)

In [ ]:
json_file = open('bodypart/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

## Predict on test set




Now let's evaluate predictions on the test data. Note that the CNN predicts labels for each image but labels exist at an exam-level. Thus we need to combine predictions for each image in an exam to produce an exam-level label. We will do so by simply averaging the prediction scores across all images in an exam.

In [ ]:
model.load_weights('bodypart/tcga-mguh-multilabel.h5')  #federated

In [ ]:
#Inference
IMSIZE = 256
WINDOW_LEVEL, WINDOW_WIDTH = 50, 500
def predict(model, images, imsize):
    '''
    Small modifications to data generator to allow for prediction on test data.
    '''
    test_arrays = [] 
 
    test_probas = [] 
    test_uids   = []
    for im in images: 
        dicom_file, uid = get_dicom_and_uid(im) 
        try:
            array = convert_dicom_to_8bit(dicom_file, WINDOW_WIDTH, WINDOW_LEVEL, 
                                    imsize=(imsize,imsize))
        except: 
            continue
      
        array = preprocess_input(array, mode='tf')
        test_arrays.append(array) 

        test_probas.append(model.predict(np.expand_dims(array, axis=0)))
        test_uids.append(uid)
    return test_uids, test_arrays, test_probas
                                    
uids, X, y_prob = predict(model, testList, IMSIZE)

test_pred_df = pd.DataFrame({'uid': uids, 'X': X, 'y_prob': y_prob})

In [ ]:
test_pred_df.apply(lambda row: row['y_prob'], axis=1)

In [ ]:
chest = np.stack(test_pred_df['y_prob'])[:,0][:,0]
abd = np.stack(test_pred_df['y_prob'])[:,0][:,1]
pelv = np.stack(test_pred_df['y_prob'])[:,0][:,2]

In [ ]:
plt.plot(chest)
plt.plot(abd)
plt.plot(pelv)

In [ ]:
numaveslices = 5
avepreds = []
allpreds = np.stack(test_pred_df['y_prob'])[:,0]
for idx,arr in enumerate(allpreds):
    low = int(max(0,idx-(numaveslices-1)/2))
    high = int(min(len(allpreds),idx+(numaveslices+1)/2))
    avepreds.append(np.mean(allpreds[low:high],axis=0))
    
chest = np.stack(avepreds)[:,0]
abd = np.stack(avepreds)[:,1]
pelv = np.stack(avepreds)[:,2]

In [ ]:
#averaged over 5 slices
plt.plot(chest)
plt.plot(abd)
plt.plot(pelv)

In [ ]:
def displayImages(imgs,labels):
    numimgs = len(imgs)
    plt.figure(figsize=(20,10))
    for idx,img in enumerate(imgs):
        dicom_file, uid = get_dicom_and_uid(img)
        img = convert_dicom_to_8bit(dicom_file, WINDOW_WIDTH, WINDOW_LEVEL, clip=False)
        plt.subplot("1%i%i" % (numimgs,idx+1))
        plt.imshow(img[...,0],cmap='gray')
        plt.title(labels[idx])
        plt.axis('off')

In [ ]:
#averaged over 5 slices
fig, ax1 = plt.subplots(figsize=(17,10))
ax1.set_xlabel("Slice Number", fontsize=20)
ax1.set_ylabel("Confidence", fontsize=20)
plt.xticks([0,30,60,90,120,150,180,210],fontsize=12)
plt.yticks(fontsize=12)
ax1.axvline(30,color='gray',ymax=0.1)
ax1.axvline(82,color='gray',ymax=0.1)
ax1.axvline(120,color='gray',ymax=0.1)
ax1.axvline(172,color='gray',ymax=0.1)
ax1.axvline(195,color='gray',ymax=0.1)
plt.plot(chest,linewidth=2,label="Chest")
plt.plot(abd,linewidth=2,label="Abdomen")
plt.plot(pelv,linewidth=2,label="Pelvis")
plt.legend(fontsize=16)

In [ ]:
displayImages([testList[30],testList[82],testList[120],testList[172],testList[195]],[30,82,120,172,195])